In [5]:
import random
import os
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras import backend as K
from envs.tetris_env import TetrisEnv

In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_shape=self.state_size, activation='relu'))
        model.add(Flatten())
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
        
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [14]:
def train(observatime, batch_size):
    env = TetrisEnv()
    agent = DQNAgent(env.state_size, env.action_size)
    train_loop(agent, env, observatime) 

    agent.replay(batch_size)
    print('Learning Finished')

    test(env, agent)


def train_loop(agent, env, observatime):
    done = False
    state = env.reset()
    state = np.expand_dims(state, axis=0) 
    for i in range(observatime):
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        next_state = np.expand_dims(next_state, axis=0)
        if i < 10:
            print(f"---{i}---")
            print(state)
            print(action) 
            print(reward)
            print(next_state)
            print(f"---{i}---")
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            state = env.reset()
            state = np.expand_dims(state, axis=0)
    print('Observing Finished')

def test(env, agent):
    observation = env.reset()
    state = np.expand_dims(observation, axis=0)
    done = False
    tot_reward = 0.0
    while not done:
        Q = agent.act(state)        
        action = np.argmax(Q)         
        next_state, reward, done, info = env.step(action)
        state = np.expand_dims(next_state, axis=0) 
        tot_reward += reward
    print('Game ended! Total reward: {}'.format(reward))

In [15]:
train(1000,100)

---0---
[[[0 1 0 0 0]
  [0 1 1 0 0]
  [0 0 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
2
0
[[[0 0 0 0 0]
  [0 0 1 1 0]
  [0 1 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
---0---
---1---
[[[0 0 0 0 0]
  [0 0 1 1 0]
  [0 1 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
0
0
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 1 1 0 0]
  [1 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
---1---
---2---
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 1 1 0 0]
  [1 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
2
0
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 0 0 0 0]
  [1 1 0 0 0]
  [0 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
---2---
---3---
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 0 0 0 0]
  [1 1 0 0 0]
  [0 1 0 0